# Refine your Data, save it to your Datastore, and register it as a Dataset
The point of refining your data is to create a version where everyone can easily use it.  First, load in your Workspace, Datastore and Datasets.

In [2]:
# Load Azure Libaries
from azureml.core import Datastore
from azureml.core.dataset import Dataset
from azureml.core.workspace import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azure.storage.blob import BlockBlobService
import pandas as pd
import numpy as np
import json
import os
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Retrieve your workspace by name by filling in the lower case values between double quotes
ws = Workspace.get(name="ancient-rivers-ml-workspace",
        subscription_id="47a7ec0c-37ad-428b-9114-b87ea1057632",
        resource_group="xeek-ancient-rivers")

In [4]:
# Retrieve your Datastore by name by filling in the lower case values between double quotes
datastore_name = "ancientrivers"
datastore = Datastore.get(ws, datastore_name)

In [5]:
# Retrieve your Datasets by name by filling in the lower case values between double quotes
dataset_name_train = "ancient_rivers_train_raw"
dataset_name_test  = "ancient_rivers_test_raw"

# Load Data in as Tabular Datasets
training_data = Dataset.get_by_name(ws, dataset_name_train, version='latest')
testing_data  = Dataset.get_by_name(ws, dataset_name_test, version='latest')


In [6]:
# Convert your tabular dataset to pandas data frames
testDF = testing_data.to_pandas_dataframe()
trainDF = training_data.to_pandas_dataframe()

# It's time to start working with your data!
This next step should be customized to fit each project following the eight steps listed below.<br>
You should understand your data, refine your data, and save it as a Dataset so everyone on your workspace can access it.<br>

0.  Select columns that you will use in your model and order them correctly<br>
1.  Rename Columns to be Comprehensible without Documentation<br>
2.  Assign Correct Data Types to Each Column<br>
3.  Fix Errors in the Data or Replace Errors with Nulls<br>
4.  Remove Rows with all Null Values<br>
5.  Remove Columns with all Null Values<br>
6.  Apply Business Logic to Create Simple Data Sets<br>
7.  Rename Coded Values to be Meaningful and easily Grasped<br>
8.  Round Meaningless Decimals<br>

In [7]:
# First, view your data
trainDF.head(5)

,row_id,well_id,GR,label
0,0,0,99.005617,0
1,1,0,105.563944,0
2,2,0,105.921065,0
3,3,0,113.513354,0
4,4,0,101.523783,0


In [8]:
testDF.head(5)

,row_id,well_id,GR
0,0,5000,134.943504
1,1,5000,127.004675
2,2,5000,133.159255
3,3,5000,134.411762
4,4,5000,135.748644


In [9]:
# Check the Data Types of your Data and make sure they match what you expect
print(trainDF.dtypes)

row_id       int64
well_id      int64
GR         float64
label        int64
dtype: object


In [10]:
print(testDF.dtypes)

row_id       int64
well_id      int64
GR         float64
dtype: object


In [16]:
# 0. Select columns that you intend to use in your model and order them correctly
testDFColumns = testDF[['row_id','well_id','GR']]
trainDFColumns = trainDF[['row_id','well_id','GR','label']]

In [17]:
# 1.  Rename your columns according to a standard.  I prefer camel caps.
testDFColumns.columns = ['RowID','WellID','GR']
trainDFColumns.columns =  ['RowID','WellID','GR','Label']

### Assign Correct Data Types

It is important to assign correct data types to your data and to remove values which do not fit, filling them with nulls. <br>
Do not remove rows with a few missing values at this stage, as this information may be valuable to data scientists during the data transformation stage.

Click this link for a detailed blog post on assigning data types in Python: https://pbpython.com/pandas_dtypes.html
    

In [18]:
# 2. Assign correct data types to each column
# 3. Convert incorrect data to nulls (Numeric)

# Test Data
testDFColumns['RowID'] = pd.to_numeric(testDFColumns['RowID'], errors='coerce')
testDFColumns['WellID'] = pd.to_numeric(testDFColumns['WellID'], errors='coerce')
testDFColumns['GR'] = pd.to_numeric(testDFColumns['GR'], errors='coerce')
# Train Data
trainDFColumns['RowID'] = pd.to_numeric(trainDFColumns['RowID'], errors='coerce')
trainDFColumns['WellID'] = pd.to_numeric(trainDFColumns['WellID'], errors='coerce')
trainDFColumns['GR'] = pd.to_numeric(trainDFColumns['GR'], errors='coerce')
trainDFColumns['Label'] = trainDFColumns['Label'].astype('category')

# Other Useful Python Functions
# pd.to_datetime(df[['<MyDateColumn>']], errors='coerce') for DateTime columns
# df["TrueFalseColumn"] = np.where(df["TrueFalseColumn"] == "Y", True, False) for Boolean (Yes / No, T / F) Columns



In [19]:
# 3. Convert incorrect data to nulls (Categorical and DateTime)

# Make a function(s) that converts unacceptable values to Nulls

# In this case, the only acceptable values are 0 and 1
def MyLabelColumnCategoryCheck(x):
    if x == 0:
        return 0
    elif x == 1:
        return 1
    elif x == 2:
        return 2
    elif x == 3:
        return 3
    elif x == 4:
        return 4
    else:
        return None

    
# Train Data    
trainDFColumns['Label'] = trainDFColumns['Label'].apply(MyLabelColumnCategoryCheck)

# Check Nulls
print(testDFColumns.isna().sum())
print(trainDFColumns.isna().sum())

RowID     0
WellID    0
GR        0
dtype: int64
RowID     0
WellID    0
GR        0
Label     0
dtype: int64


### Removing Null Rows and Columns

Rows that are completely Null and Columns that are completely Null hold no value and should be removed.

Click this link for a detailed blog post on Null removal in Python: https://www.journaldev.com/33492/pandas-dropna-drop-null-na-values-from-dataframe
    

In [23]:
# 4. Remove Rows with all Null Values

# Test Data
testDFNullRemoved = testDFColumns.dropna(how='all')
# Train Data
trainDFNullRemoved = trainDFColumns.dropna(how='all')

# 5. Remove Columns with all Null Values

# Test Data
testDFNulLRemoved = testDFNullRemoved.dropna(how='all', axis = 1)
# Train Data
trainDFNullRemoved = trainDFNullRemoved.dropna(how='all', axis = 1)


### Apply Business Logic to Make Simplified Data Sets

Each data set is going to have its own business logic that can get very confusing.  It is important that new employees can grasp the meaning of your Data.<br>  In order to do so, follow the advice below: <br>
<br>
1.  If Business Months, Weeks and Years do not correspond to their calendar dates, create new columns to capture that information.<br>
2.  If a single column contains multiple measurements, such as centimeters and inches, create a column to capture that information.<br>
3.  If a table contains multiple grains of data, such as rows containing aggregates for both day and month, split the table by grain into new tables.<br>
4.  If the same value in the same column has multiple meanings based on a second column, change the original column to have different values. <br>
5.  Rearrange the data so that most questions can be answered with simple queries.<br>For example, you should always be able to get an accurate sales number by summing the sales column without involving a second column.<br>
6.  There are many other similar transformations you should do, always with the goal of simplicity.<br>
7.  Test your data by showing it to someone who knows SQL but not your data and ask them to solve simple business questions.<br>

In [24]:
# 6.  Apply Business Logic 

# Apply your own logic here

# Test Data
testDFLogicApplied = testDFNullRemoved
# Train Data
trainDFLogicApplied = trainDFNullRemoved

In [25]:
# 7. Rename coded values to be meaningful and easily grasped

# Make a function that converts coded values to readable values

# In this case, we assign names to the label which correspond to sediment layer types
def MyLabelColumnCodeReader(x):
    if x == 0:
        return 'Default Layer'
    elif x == 1:
        return 'Layer 1'
    elif x == 2:
        return 'Layer 2'
    elif x == 3:
        return 'Layer 3'
    elif x == 4:
        return 'Layer 4'
    else:
        return None

    
# Train Data
trainDFLogicApplied['Label']  = trainDFLogicApplied['Label'].apply(MyLabelColumnCodeReader)

In [26]:
# 8. Round Meaningless Decimals.
# Meaningless Decimals cost companies a lot of money.  Round them to save cash and compute time.

# Round each of your numerical columns.  In this case, we will round to 1 decimal.

# Test Data
testRefinedDF = testDFLogicApplied.round({'GR': 1})
# Train Data
trainRefinedDF = trainDFLogicApplied.round({'GR': 1})


### Write your refined data back to your Data Lake
After refining your data, save it to a place on a data lake where it is accessible to all of the data workers who require access.<br>
Make sure that you indicate in the folder path and file name that it is refined, curated data.

In [27]:
# Output a nice table with all of your essential datastore information so you have it on hand
dsoutput = {}
dsoutput['Workspace Name'] = ws.name
dsoutput['Datastore Name'] = datastore.name
dsoutput['Container Name'] = datastore.container_name
dsoutput['Resource Group'] = ws.resource_group
dsoutput['Storage Account'] = datastore.account_name
pd.set_option('display.max_colwidth', -1)
dsoutputDf = pd.DataFrame(data = dsoutput, index = [''])
dsoutputDf.T

,
Workspace Name,ancient-rivers-ml-workspace
Datastore Name,ancientrivers
Container Name,dms-data-lake
Resource Group,xeek-ancient-rivers
Storage Account,dmsdatalake


In [28]:
# Test Data

# Create the BlockBlockService that the system will use to write data.  
# Specify your datastore storage account and account key.
block_blob_service = BlockBlobService(
    account_name='dmsdatalake', account_key='pu0OWdEM2rqecJBhIZoLqPMo/DLl3JQQ3eryF1HJVbejjLyUFqoOlVIQ4rv24v+1xlJZjRv9x1Ix4ucTqZqWKw==')

# Specify your container in your storage account where you wish to save Refined Data.
# It can be different from the container used in your primary datastore, in which case register it as another datastore
container_name = 'dms-data-lake'

# Change your Pandas Dataframe to CSV
testRefinedCSV = testRefinedDF.to_csv(index=False)

# Assign a path and filename inside your container in your storage account
data_lake_file_path =  "dev/refined/test/ancient-rivers-test"

# Upload the CSV into your storage account
block_blob_service.create_blob_from_text(
    container_name, data_lake_file_path,testRefinedCSV)

In [29]:
# Train Data

# Create the BlockBlockService that the system will use to write data.  
# Specify your datastore storage account and account key.
block_blob_service = BlockBlobService(
    account_name='dmsdatalake', account_key='pu0OWdEM2rqecJBhIZoLqPMo/DLl3JQQ3eryF1HJVbejjLyUFqoOlVIQ4rv24v+1xlJZjRv9x1Ix4ucTqZqWKw==')

# Specify your container in your storage account where you wish to save Refined Data.
# It can be different from the container used in your primary datastore, in which case register it as another datastore
container_name = 'dms-data-lake'

# Change your Pandas Dataframe to CSV
trainRefinedCSV = trainRefinedDF.to_csv(index=False)

# Assign a path and filename inside your container in your storage account
data_lake_file_path =  "dev/refined/train/ancient-rivers-train"

# Upload the CSV into your storage account
block_blob_service.create_blob_from_text(
    container_name, data_lake_file_path,trainRefinedCSV)

### Register your Refined Datasets
This allows you to share your Curated Datasets with others in your workspace, to version and keep track of them.

In [32]:
# If you stored your Refined Datasets in a separate container, first create another Datastore for that container.

In [33]:
# Specify the files and/or directories in your datastore.  You can pull in multiple or single files.
# Here, we pull in testing data and training data separately

datastore_path_refined_test = [
                  (datastore,  "dev/refined/test/ancient-rivers-test")
                 ]

datastore_path_refined_train = [
                  (datastore,  "dev/refined/train/ancient-rivers-train")
                 ]

In [34]:
# Create Tabular Data Sets
test_data_refined = Dataset.Tabular.from_delimited_files(path=datastore_path_refined_test)
train_data_refined = Dataset.Tabular.from_delimited_files(path=datastore_path_refined_train)

In [35]:
# Register and tag your data sets
test_data_refined.register(workspace=ws,
                        name="ancient-rivers-test-refined",
                        description="Curated Ancient Rivers Testing Data.  No Label Included",
                        tags = {"Type": "Classification", "Project": "Ancient Rivers"},
                        create_new_version=True)

{
  "source": [
    "('ancientrivers', 'dev/refined/test/ancient-rivers-test')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "d922ed97-3d6e-45b7-b804-fc7c3b8594b7",
    "name": "ancient-rivers-test-refined",
    "version": 1,
    "description": "Curated Ancient Rivers Testing Data.  No Label Included",
    "tags": {
      "Type": "Classification",
      "Project": "Ancient Rivers"
    },
    "workspace": "Workspace.create(name='ancient-rivers-ml-workspace', subscription_id='47a7ec0c-37ad-428b-9114-b87ea1057632', resource_group='xeek-ancient-rivers')"
  }
}

In [36]:
train_data_refined.register(workspace=ws,
                        name="ancient-rivers-train-refined",
                        description="Curated Ancient Rivers Training Data.  Label Included",
                        tags = {"Type": "Classification", "Project": "Ancient Rivers"},
                        create_new_version=True)

{
  "source": [
    "('ancientrivers', 'dev/refined/train/ancient-rivers-train')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "8f424c06-72ca-47d8-9c8d-2dffd629da73",
    "name": "ancient-rivers-train-refined",
    "version": 1,
    "description": "Curated Ancient Rivers Training Data.  Label Included",
    "tags": {
      "Type": "Classification",
      "Project": "Ancient Rivers"
    },
    "workspace": "Workspace.create(name='ancient-rivers-ml-workspace', subscription_id='47a7ec0c-37ad-428b-9114-b87ea1057632', resource_group='xeek-ancient-rivers')"
  }
}